In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Imports

In [ ]:
import os
import json

import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import cv2
import albumentations as A
from sklearn import metrics as sk_metrics
from skimage.io import imread
import tensorflow as tf
from sklearn.model_selection import train_test_split 

# Nombres Directorios

In [ ]:
BASE_DIR = "../input/cassava-leaf-disease-classification/"
TRAIN_PATH = BASE_DIR + "train_images/"

# Direccion

In [ ]:
with open(os.path.join(BASE_DIR, "label_num_to_disease_map.json")) as file:
    map_classes = json.loads(file.read())
    map_classes = {int(k) : v for k, v in map_classes.items()}
    
print(json.dumps(map_classes, indent=4))

In [ ]:
input_files = os.listdir(os.path.join(BASE_DIR, "train_images"))
print(f"Number of train images: {len(input_files)}")

In [ ]:
datos = pd.read_csv(BASE_DIR+"train.csv")
datos

In [ ]:
datos.groupby('label')['image_id'].count()

# Cargar las fotos

In [ ]:
IMAGE_WIDTH = IMAGE_HEIGHT = 64
def cargar_fotos(directorio):
    X = []
    Y = []
    for image in os.listdir(directorio):
        foto = imread(directorio+image)
        smallimage = cv2.resize(foto, (IMAGE_WIDTH, IMAGE_HEIGHT)) 
        X.append(smallimage) 
        category = datos[datos["image_id"] == image]['label'].values[0]
        Y.append(category)

    return np.array(X), np.array(Y)

In [ ]:
X_train, y_train = cargar_fotos(TRAIN_PATH)

In [ ]:
plt.imshow(X_train[0]);

# Estandarizar

In [ ]:
# solo ejecutar una vez
print("Min:", np.min(X_train))
print("Max:", np.max(X_train))

X_train = X_train / 255.0

print("Min:", np.min(X_train))
print("Max:", np.max(X_train))

# Shuffle

In [ ]:
from sklearn.utils import shuffle 

X_train, y_train = shuffle(X_train, y_train, random_state = 42)

# Separar datos en train y test

In [ ]:
X = X_train
y = y_train

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modelos

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3),   
                        activation = 'relu',
                       input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT, 3)),  
    
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(64, (3,3),
                       activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dense(5, activation = 'softmax') 
    
])

In [ ]:
model.compile(optimizer = 'adam',      # que funcion de coste minimizar
             loss = 'sparse_categorical_crossentropy', #sparse_categorical_crossentrophy si tuvieramos mas
              metrics = ['accuracy'])

In [ ]:
from keras.callbacks import EarlyStopping
earlystop = EarlyStopping(patience=2)

In [ ]:
history = model.fit(X_train,
                   y_train,
                   epochs = 5,
                    callbacks = [earlystop],
                   validation_split = 0.2)

In [ ]:
results = model.evaluate(X_test, y_test)
print("test loss, test acc:", results)